In [1]:
import os
os.chdir(r'C:\Users\Anil\OneDrive\Desktop\data\json')

In [2]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark'

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [49]:
df = spark.read.json("C:/Users/Anil/*json")

In [5]:
print(df.count())
print(df.printSchema())

290
root
 |-- actor: struct (nullable = true)
 |    |-- displayName: string (nullable = true)
 |    |-- favoritesCount: long (nullable = true)
 |    |-- followersCount: long (nullable = true)
 |    |-- friendsCount: long (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- image: string (nullable = true)
 |    |-- languages: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- link: string (nullable = true)
 |    |-- links: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- rel: string (nullable = true)
 |    |-- listedCount: long (nullable = true)
 |    |-- location: struct (nullable = true)
 |    |    |-- displayName: string (nullable = true)
 |    |    |-- objectType: string (nullable = true)
 |    |-- objectType: string (nullable = true)
 |    |-- postedTime: timestamp (nullable = true)
 |    |-- preferredUsername: string (nullable = true)

In [50]:
df = df.select([c for c in df.columns if c in ['body','gnip', 'id', 'postedTime', 'twitter_entities', 'twitter_lang']])
df = df.withColumn("country_code", (df.gnip.matching_rules.tag[0]))
df = df.filter(df.country_code.contains('US'))
df = df.filter(df['twitter_lang'] == 'en')

In [51]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import split, regexp_replace
from pyspark.sql.functions import col
import re

def processTweet(tweet):
    #Remove handles
    tweet = ' '.join(re.sub("\s([@#][\w_-]+)"," ",tweet).split())
    #Remove HTML special entities (e.g. &amp;)
    tweet = re.sub(r'\&\w*;', '', tweet)
    #Remove tickers
    tweet = re.sub(r'\$\w*', '', tweet)
    #To lowercase
    tweet = tweet.lower()
    #Remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet)
    #Remove Punctuation and split 's, 't, 've with a space for filter
    tweet =  re.sub('[^.,a-zA-Z0-9 \n\.]', '',tweet)
    #Remove whitespace (including new line characters)
    tweet = re.sub(r'\s\s+', ' ', tweet)
    #Remove single space remaining at the front of the tweet.
    tweet = tweet.lstrip(' ') 
    # Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    tweet = ''.join(c for c in tweet if c <= '\uFFFF') 
    return tweet

udfprocessTweet = udf(processTweet, StringType())
df = df.withColumn("text", udfprocessTweet("body"))

In [52]:
df.show(10)

+--------------------+--------------------+--------------------+-------------------+--------------------+------------+------------+--------------------+
|                body|                gnip|                  id|         postedTime|    twitter_entities|twitter_lang|country_code|                text|
+--------------------+--------------------+--------------------+-------------------+--------------------+------------+------------+--------------------+
|TART Green Apple ...|[[[42383907434077...|tag:search.twitte...|2017-01-01 05:30:53|[[],, [], [[fb.me...|          en|         US1|tart green apple ...|
|If you drink #cof...|[[[12518545021641...|tag:search.twitte...|2017-01-01 05:30:54|[[[[13, 20], coff...|          en|         US2|if you drink righ...|
|Pumpkin Protein S...|[[[12518545021641...|tag:search.twitte...|2017-01-01 05:30:35|[[],, [], [[pinte...|          en|         US2|pumpkin protein s...|
|Drinking a Dragon...|[[[42383907434077...|tag:search.twitte...|2017-01-01 05:30:1

In [ ]:
#######################################
#Model

In [1]:
####################################Inclusions###################################
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, ArrayType


inclusions = pd.read_excel(r"C:\Users\Anil\OneDrive\Desktop\data\Inclusion_Terms.xlsx")
#inclusions = inclusions.drop(["Russian" , "Kazakh", "S. No", "Portugese"], axis = 1)
inclusions['Primary Terms'] =  inclusions['Primary Terms'].str.lower()

sdf_inclusion = spark.createDataFrame(inclusions)

inclusion = [row['Primary Terms'] for row in sdf_inclusion.collect()]
def intersection_function(list1):
    list1 = list1.split()# if value in list1]
    intersection_list = list(set(inclusion).intersection(list1))
    return intersection_list

udf_intersection = F.udf(intersection_function, StringType())

df = df.withColumn("Matched_Inclusion", udf_intersection(df["text"]))

NameError: name 'spark' is not defined

In [10]:
sdf_inclusion.show()

+---------------+
|  Primary Terms|
+---------------+
|          dairy|
|           milk|
|   sports drink|
|   energy drink|
|       lemonade|
| packaged water|
|   spring water|
| alkaline water|
|  infused water|
|flavoured water|
+---------------+



In [54]:
#################################### Matched Primary ###################################
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, ArrayType


inclusions = pd.read_excel(r"C:\Users\Anil\OneDrive\Desktop\\data\Inclusion_Primary.xlsx")
#inclusions = inclusions.drop(["Russian" , "Kazakh", "S. No", "Portugese"], axis = 1)
inclusions['Primary Terms'] =  inclusions['Primary Terms'].str.lower()

sdf_inclusion = spark.createDataFrame(inclusions)

inclusion = [row['Primary Terms'] for row in sdf_inclusion.collect()]
def intersection_function(list1):
    list1 = list1.split()# if value in list1]
    intersection_list = list(set(inclusion).intersection(list1))
    return intersection_list

udf_intersection = F.udf(intersection_function, StringType())

df = df.withColumn("Matched_Primary", udf_intersection(df["text"]))

In [13]:
import string
import pandas as pd
import numpy as np
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

# clean text data
udfprocessNoun = udf(clean_text, StringType())
df = df.withColumn("text1", udfprocessNoun("text"))

PicklingError: args[0] from __newobj__ args has the wrong class

In [55]:
df.show(5)

+--------------------+--------------------+--------------------+-------------------+--------------------+------------+------------+--------------------+-----------------+---------------+
|                body|                gnip|                  id|         postedTime|    twitter_entities|twitter_lang|country_code|                text|Matched_Inclusion|Matched_Primary|
+--------------------+--------------------+--------------------+-------------------+--------------------+------------+------------+--------------------+-----------------+---------------+
|TART Green Apple ...|[[[42383907434077...|tag:search.twitte...|2017-01-01 05:30:53|[[],, [], [[fb.me...|          en|         US1|tart green apple ...|          [dairy]|        [dairy]|
|If you drink #cof...|[[[12518545021641...|tag:search.twitte...|2017-01-01 05:30:54|[[[[13, 20], coff...|          en|         US2|if you drink righ...|               []|             []|
|Pumpkin Protein S...|[[[12518545021641...|tag:search.twitte...|2

In [28]:
from textblob import TextBlob
def sentiment(x):
    sentiment = TextBlob(x)
    return sentiment.sentiment.polarity

udfprocess_sentiment = udf(getSentiment, FloatType())
df = df.withColumn("text1", udfprocess_sentiment("text"))

In [58]:
#!pip install textblob
from textblob import TextBlob
import re
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf

def sentiment_score(chat):
    return TextBlob(chat).sentiment.polarity

predictions = udf(sentiment_score)
# df.withColumn("predicted", predictions(df['Text'])).show(5)

blob_df = df.withColumn("predicted", predictions(df["Text"]))
blob_df.show(5)

#blob_df.show(5)



+--------------------+--------------------+--------------------+-------------------+--------------------+------------+------------+--------------------+-----------------+---------------+--------------------+
|                body|                gnip|                  id|         postedTime|    twitter_entities|twitter_lang|country_code|                text|Matched_Inclusion|Matched_Primary|           predicted|
+--------------------+--------------------+--------------------+-------------------+--------------------+------------+------------+--------------------+-----------------+---------------+--------------------+
|TART Green Apple ...|[[[42383907434077...|tag:search.twitte...|2017-01-01 05:30:53|[[],, [], [[fb.me...|          en|         US1|tart green apple ...|          [dairy]|        [dairy]|                -0.2|
|If you drink #cof...|[[[12518545021641...|tag:search.twitte...|2017-01-01 05:30:54|[[[[13, 20], coff...|          en|         US2|if you drink righ...|               [

In [37]:
text1 = TextBlob("Today is a great day, but it is boring")
text1.sentiment.polarity

-0.09999999999999998

In [29]:
df.show(5)

Py4JJavaError: An error occurred while calling o460.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 10, MSI.mshome.net, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 597, in main
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 589, in process
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 465, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 454, in _batched
    for item in iterator:
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 442, in mapper
  File "C:\spark\python\lib\pyspark.